In [5]:
# %load ../nash-equi/utils.py
import numpy as np
import itertools

def support(sigma, tol=1e-10):
    """
    get support of a strategy
    """
    sigma = np.array(sigma)
    return np.where(sigma > tol)[0]

def is_best_response(A, sigma_A, sigma_B):
    """
    check if a given strategy sigma_A is the best response to opponent
    strategy sigma_B
    """
    # set as np.arrays
    sigma_A, sigma_B = np.array(sigma_A), np.array(sigma_B)
    spt = support(sigma_A)
    best = True
    Ay = A @ sigma_B
    max_Ay = max(Ay)
    for i in spt:
        if Ay[i] != max_Ay:
            best = False
    return best

def indifference(payoff, support_u, support_v):
    """
    compute indifference matrix for a given support.
    for reference, just check out "algorithmic game theory"
    by nisan et. al
    """
    n = len(payoff[0])
    M = np.zeros((len(support_u) + n - len(support_v), n), dtype=np.float64)
    # condition 1: best response condition for given support
    for i in range(len(support_u) - 1):
        M[i] = payoff[support_u[i]] - payoff[support_u[i+1]]
    # condition 2: ensure support is in S(v)
    complement_Sv = [j for j in range(n) if j not in support_v]
    for k, i in enumerate(range(len(support_u) - 1, len(support_u) + n - len(support_v) - 1)):
        M[i][complement_Sv[k]] = 1
    # condition 3: last row is all 1's to ensure solution is a probability vector
    M[-1] = np.ones(n)
    return M

def equilibrium_candidate(payoff, support_u, support_v):
    """
    using indifference, compute a candidate for the nash equilibrium
    given fixed supports
    """
    M = indifference(payoff, support_u, support_v)
    b = np.zeros(M.shape[1])
    b[-1] = 1
    # return np.linalg.inv(M) @ b
    return np.linalg.solve(M, b)

def get_powerset(n):
    """
    get powerset of n using itertools
    """
    return itertools.chain.from_iterable(
        itertools.combinations(range(n), r) for r in range(n + 1))

In [6]:
# %load ../nash-equi/game.py
"""
stupid implementation of a normal form game, and eventual
computations of nash equilibria
"""

import numpy as np

class game:
    """
    2 player normal form game
    """
    def __init__(self, payoff_A, payoff_B=None):
        self.payoff_A = np.array(payoff_A)
        if payoff_B is None:
            # zero-sum game
            self.payoff_B = -self.payoff_A
        else:
            self.payoff_B = np.array(payoff_B)
        
    def __repr__(self):
        return "player 1 payoff:\n" + str(self.payoff_A) + "\n\n" + \
               "player 2 payoff:\n" + str(self.payoff_B)
    
    def payoff(self, sigma_1, sigma_2):
        expected_payoff_1 = sigma_1.T @ self.payoff_A @ sigma_2
        expected_payoff_2 = sigma_1.T @ self.payoff_B @ sigma_2
        return np.array([expected_payoff_1, expected_payoff_2])

    def equilibria(self):
        """
        find all Nash equilibria of 2-player normal form game via 
        support enumeration.
        """
        n, m = self.payoff_A.shape
        # create supports
        for support_u in (s for s in get_powerset(n) if len(s) > 0):
            for support_v in (s for s in get_powerset(m) if len(s) == len(support_u)):
                # get candidate for nash equilibrium
                try:
                    candidate_A = equilibrium_candidate(self.payoff_A, support_u, support_v)
                    candidate_B = equilibrium_candidate(self.payoff_B.T, support_v, support_u)
                    # check if best response to one another
                    if is_best_response(self.payoff_B.T, candidate_A, candidate_B) and \
                        is_best_response(self.payoff_A, candidate_B, candidate_A):
                            return [candidate_A, candidate_B]
                except:
                    continue
        return "no Nash equilibria found"

In [7]:
A = [[1, 1, -1], [2, -1, 0]]
B = [[1/2, -1, -1/2], [-1, 3, 2]]

g = game(A, B)
g

player 1 payoff:
[[ 1  1 -1]
 [ 2 -1  0]]

player 2 payoff:
[[ 0.5 -1.  -0.5]
 [-1.   3.   2. ]]

In [8]:
g.equilibria()

[array([-0.        ,  0.33333333,  0.66666667]),
 array([0.66666667, 0.33333333])]

In [9]:
# coin flipping
A = [[1, -1], [-1, 1]]
B = [[-1, 1], [1, -1]]

g = game(A, B)
g

player 1 payoff:
[[ 1 -1]
 [-1  1]]

player 2 payoff:
[[-1  1]
 [ 1 -1]]

In [10]:
g.equilibria()

[array([0.5, 0.5]), array([0.5, 0.5])]

In [11]:
# rock paper scissors
A = [[0, -1, 1], [1, 0, -1], [-1, 1, 0]]

rps = game(A)
rps

player 1 payoff:
[[ 0 -1  1]
 [ 1  0 -1]
 [-1  1  0]]

player 2 payoff:
[[ 0  1 -1]
 [-1  0  1]
 [ 1 -1  0]]

In [14]:
rps.equilibria()

[array([0.33333333, 0.33333333, 0.33333333]),
 array([0.33333333, 0.33333333, 0.33333333])]